In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
filepath = Path("../sample_files/ellipsoid_2.stl")
lv = LV.from_file(filepath)
lv.tetrahedralize(backend=TETRA_BACKEND.TETGEN, quality=True, order=1, mindihedral=10, minratio=1.4, nobisect=True)
lv.plot(show_edges=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(position=(49999.999999…

In [3]:
from project_heart.utils.spatial_utils import *
from project_heart.utils.vector_utils import *

In [4]:
lv.identify_base_and_apex_surfaces()
lv.identify_epi_endo_surfaces(90)

(array([2, 2, 2, ..., 2, 2, 2]), array([2, 2, 2, ..., 0, 0, 0]))

In [5]:
lv.plot()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(position=(49999.999999…

In [6]:
e1 = 45
b1 = 1
b2 = 89

endo_epi = np.copy(lv.get(GEO_DATA.SURF_POINT_DATA, LV_MESH_DATA.EPI_ENDO_GUESS))
lvsurf = lv.get_surface_mesh()
# Get edges
edges = lv.mesh.extract_feature_edges(e1)
edge_pts = edges.points
est_base = centroid(edge_pts)
est_radius = radius(edge_pts)
# select pts close to est_base based on dist threshold on axis
pts = lvsurf.points
lvnormal = lv.get_normal()
axis = np.where(lvnormal==np.max(lvnormal))[0]
d_base = np.abs(est_base[axis] - pts[:, axis])
ioi = np.where(d_base <= 0.1)[0] 
# print(ioi)
# filter selected pts based on surface angle
surfnormals = lvsurf.compute_normals(inplace=True)
surf_normals = lvsurf.get_array("Normals", "points")
# base_angles = lv.compute_angles_wrt_normal(surf_normals[ioi], check_orientation=False, degrees=False)
len(ioi)
zvec = np.repeat(np.expand_dims(lv._Z, 1), len(ioi), axis=1).T
base_angles = angle_between(surf_normals[ioi], zvec, check_orientation=False)
ioi = ioi[np.where(base_angles <= np.radians(90))[0]]
print(len(ioi))
# filter by endo
ioi = ioi[np.where(endo_epi[ioi] != LV_SURFS.ENDO)]
print(len(ioi))

# identify final surfaces
endo_epi_base = np.copy(endo_epi)
endo_epi_base[ioi] = LV_SURFS.BASE
# map to 'global' mesh ids
id_map = lv.get_surface_id_map_from_mesh()
endo_epi_base_mesh = np.zeros(lv.mesh.n_points)
endo_epi_base_mesh[id_map] = endo_epi_base
# add data to mesh
lv._surface_mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base
lv.mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base_mesh

lv.transform_point_data_to_cell_data(LV_MESH_DATA.SURFS, method=lambda x: int(np.max(x)))
#plot
lv.plot("mesh", 
        LV_MESH_DATA.SURFS.value,
        show_edges=True,
        vnodes=[
          (LV_VIRTUAL_NODES.APEX, "green"),
          (LV_VIRTUAL_NODES.BASE, "red"),
          ],
          cmap="Set1",
          
        categorical=True,
        split_sharp_edges=True,
        smooth_shading=False)

960
960


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(position=(49999.999999…

In [26]:
lv.compute_fibers(surfRegionsIds=LV_MESH_DATA.SURFS,
                    fiber_space="P_1",
                    alpha_endo_lv=60,  # Fiber angle on the endocardium
                    alpha_epi_lv=-60,  # Fiber angle on the epicardium
                    beta_endo_lv=0,  # Sheet angle on the endocardium
                    beta_epi_lv=0,  # Sheet angle on the epicardium
                    markers={
                      "lv": LV_SURFS.ENDO.value,
                      "epi": LV_SURFS.EPI.value,
                      "base": LV_SURFS.BASE.value
                    },
                    ldrb_kwargs={},
                    save_xdmfs=False,
                    xdmfs_dir=None,
                    xdmfs_basename=None,
                    del_generated_files=True,
                    )

Calculating scalar fields
Compute scalar laplacian solutions with the markers: 
lv: 1
epi: 2
base: 23
  Num vertices: 37027
  Num cells: 172466
Solving linear variational problem.
  Apex coord: (-0.00, 0.00, -75.09)
Solving linear variational problem.
Solving Laplace equation
 lv = 1, epi = 0
Solving linear variational problem.
 epi = 1, lv = 0
Solving linear variational problem.

Calculating gradients
Compute fiber-sheet system
Angles: 
alpha: 
 endo_lv: 60
 epi_lv: -60
 endo_septum: 60
 epi_septum: -60
 endo_rv: 60
 epi_rv: -60
beta: 
 endo_lv: 0
 epi_lv: 0
 endo_septum: 0
 epi_septum: 0
 endo_rv: 0
 epi_rv: 0


In [27]:
lv.regress_fibers(lv, hidden_layer_sizes=(100,50), verbose=True, max_iter=250, validation_fraction=0.25, tol=1e-5)

Iteration 1, loss = 0.28545223
Validation score: 0.303710
Iteration 2, loss = 0.08565729
Validation score: 0.404121
Iteration 3, loss = 0.07881783
Validation score: 0.450724
Iteration 4, loss = 0.07391617
Validation score: 0.478599
Iteration 5, loss = 0.06975152
Validation score: 0.505895
Iteration 6, loss = 0.06551950
Validation score: 0.523046
Iteration 7, loss = 0.06064537
Validation score: 0.550857
Iteration 8, loss = 0.05592967
Validation score: 0.574339
Iteration 9, loss = 0.05037324
Validation score: 0.595915
Iteration 10, loss = 0.04517935
Validation score: 0.619095
Iteration 11, loss = 0.04077030
Validation score: 0.650642
Iteration 12, loss = 0.03523526
Validation score: 0.682956
Iteration 13, loss = 0.03133837
Validation score: 0.714348
Iteration 14, loss = 0.02701251
Validation score: 0.741089
Iteration 15, loss = 0.02379312
Validation score: 0.762254
Iteration 16, loss = 0.02152651
Validation score: 0.778910
Iteration 17, loss = 0.01983742
Validation score: 0.786060
Iterat

array([[ 0.10805818,  0.88699898, -0.4082346 , ...,  0.38337392,
        -0.48673593, -0.81940762],
       [ 0.01615882,  0.89570597, -0.42053608, ...,  0.40452275,
        -0.43766554, -0.82802809],
       [ 0.02589543,  0.90389808, -0.43042411, ...,  0.4197937 ,
        -0.44997538, -0.81326126],
       ...,
       [-0.39359802,  0.60125576,  0.70882345, ...,  0.29312203,
         0.78794692, -0.55687746],
       [-0.03452201, -0.06980684,  0.13926463, ..., -0.65312877,
        -0.21713846, -0.6317865 ],
       [-0.33004997,  0.56026192,  0.80051194, ...,  0.67541091,
         0.6936906 , -0.15544349]])

In [28]:
lv.transform_point_data_to_cell_data(LV_FIBERS.F0, "mean", axis=0)
lv.transform_point_data_to_cell_data(LV_FIBERS.S0, "mean", axis=0)
lv.transform_point_data_to_cell_data(LV_FIBERS.N0, "mean", axis=0)

pyvista_ndarray([[-0.0264534 , -0.51143103,  0.3993329 ],
                 [ 0.37132558,  0.69282043,  0.15911942],
                 [-0.46726525, -0.21092379,  0.84648245],
                 ...,
                 [-0.27279385,  0.45800466,  0.83966439],
                 [-0.28093073,  0.45090226,  0.84086057],
                 [-0.27063527,  0.45770474,  0.83950582]])

In [29]:
lv.mesh.save("./test3.vtk")

In [ ]:
lv.plot_streamlines(LV_FIBERS.F0, decimate_boundary=0.1)

In [ ]:
lv.mesh.save("./test2.vtk")